## Load Routines for Printing Matrices

In [1]:
      
include("printmat.jl")

println4Ps (generic function with 1 method)

# Exchange Rate Quotations

In [2]:
CHF_USD = 0.9970
USD_AUD = 0.8139

CHF_AUD = CHF_USD*USD_AUD

printlnPs("CHF per AUD: ",CHF_AUD)

CHF per AUD:      0.811


# Currency Returns

The strategy is: in $t=0$, you borrow on the domestic money market (at the safe rate Rf), buy foreign currency (at the price $S_0$) at lend on foreign money market (at the safe rate Rfstar). Finally, in $t=1$ sell the foreign currency (at the price $S_1$). 

Notice that Rf and Rfstar are the safe rates over the investment period (for instance, one-month period). Conversion from annualized interest rates to these monthly rates is discussed under UIP (below).

In [3]:
S₀ = 1.2
S₁ = 1.25
Rfstar = 0.03
Rf     = 0.02

Re = (1+Rfstar)*S₁/S₀ - 1 - Rf
printlnPs("Currency excess return (from t=0 to t=1)")
printlnPs("if S₁ = $S₁: ",Re)

S₁ = 1.189
Re = (1+Rfstar)*S₁/S₀ - 1 - Rf
printlnPs("if S₁ = $S₁: ",Re)

S₁ = 1.177
Re = (1+Rfstar)*S₁/S₀ - 1 - Rf
printlnPs("if S₁ = $S₁: ",Re)

Currency excess return (from t=0 to t=1)
if S₁ = 1.25:      0.053
if S₁ = 1.189:      0.001
if S₁ = 1.177:     -0.010


# Uncovered Interest Rate Parity (UIP)


UIP assumes that the expected future exchange rate (E$_0S_m$) is fully determined by the current (as of $t=0$) exchange rate and interest rates, in such a way that the expected excess return on a a foreign investment (on the foreign money market) is zero.

Interest rates are typically annualized (denoted $y$ and $y^*$ below). This means that the safe rate over an investment period of $m$ years (eg. $m=1/12$ for a month) is $(1+y)^m$.

In [4]:
S₀ = 1.2
y  = 0.04
ystar = 0.06
m = 1/2
ESm = S₀ * (1+y)^m/(1+ystar)^m

printlnPs("Expected future exchange rate according to UIP: ",ESm)

Expected future exchange rate according to UIP:      1.189


In [5]:
Sm = 1.15        #suppose the exchange rate ends up at this value

grossR_foreign  = Sm/S₀*(1+ystar)^m
grossR_domestic = (1+y)^m

println("\nGross returns on foreign and domestic investments, difference: ")
printmat([grossR_foreign grossR_domestic grossR_foreign-grossR_domestic])


Gross returns on foreign and domestic investments, difference: 
     0.987     1.020    -0.033

